In [2]:
import pandas as pd

# Data Cleaning

In [13]:
df_historical_data = pd.read_csv('data/fifa_worldcup_historical_data.csv')
df_fixture = pd.read_csv('data/fifa_worldcup_fixture.csv')
df_missing_data = pd.read_csv('data/fifa_worldcup_missing_data.csv')

## Cleaning df_fixture

In [14]:
df_fixture.head()

,home,away,score,year
0,Qatar,Ecuador,0–2,2022
1,Senegal,Netherlands,0–2,2022
2,Qatar,Senegal,Match 18,2022
3,Netherlands,Ecuador,Match 19,2022
4,Ecuador,Senegal,Match 35,2022


In [15]:
df_fixture['home'] = df_fixture['home'].str.strip()
df_fixture['away'] = df_fixture['away'].str.strip()

## Cleaning df_missing_data and adding it to df_historical_data

In [16]:
# Missing Data
df_missing_data=df_missing_data[df_missing_data['year']!=2010]
df_missing_data.head()

,home,score,away,year
0,Mexico,0–0,Soviet Union,1970
1,Belgium,3–0,El Salvador,1970
2,Soviet Union,4–1,Belgium,1970
3,Mexico,4–0,El Salvador,1970
4,Soviet Union,2–0,El Salvador,1970


In [11]:
# df_missing_data[df_missing_data['home'].isnull()]
df_missing_data.dropna(inplace=True)
df_historical_data = pd.concat([df_historical_data, df_missing_data], ignore_index=True)
df_historical_data.drop_duplicates(inplace=True)
df_historical_data.sort_values('year', inplace=True)
df_historical_data

,home,away,score,year
0,France,Mexico,4–1,1930
17,Uruguay,Argentina,4–2,1930
16,Uruguay,Yugoslavia,6–1,1930
15,Argentina,United States,6–1,1930
14,Paraguay,Belgium,1–0,1930
...,...,...,...,...
489,Brazil,Costa Rica,2–0,2018
490,Serbia,Switzerland,1–2,2018
491,Serbia,Brazil,0–2,2018
493,Germany,Mexico,0–1,2018


In [18]:
df_historical_data[df_historical_data['home'].isnull()]

,home,away,score,year


## Cleaning df_historical_data

In [19]:
# There was only one match that one of the team didn't show, we will delete that match beacuse it don't represent a score based on the game.
# deleting match with walk over
delete_index = df_historical_data[df_historical_data['home'].str.contains('Sweden') &
                                    df_historical_data['away'].str.contains('Austria')].index

df_historical_data.drop(index=delete_index, inplace=True)

In [21]:
# We 'll eliminate text values from the score column -> (a.e.t.)
df_historical_data[df_historical_data['score'].str.contains('[^\d–]')]

,home,away,score,year
24,Austria,France,3–2 (a.e.t.),1934
27,Italy,Spain,1–1 (a.e.t.),1934
34,Italy,Czechoslovakia,2–1 (a.e.t.),1934
35,Switzerland,Germany,1–1 (a.e.t.),1938
38,Cuba,Romania,3–3 (a.e.t.),1938
...,...,...,...,...
513,Spain,Russia,1–1 (a.e.t.),2018
514,Croatia,Denmark,1–1 (a.e.t.),2018
518,Colombia,England,1–1 (a.e.t.),2018
522,Russia,Croatia,2–2 (a.e.t.),2018


In [22]:
# cleanning score and home/away columns
df_historical_data['score'] = df_historical_data['score'].str.replace('[^\d–]', '', regex=True)
# Cleaning blank spaces in home and away
df_historical_data['home'] = df_historical_data['home'].str.strip()
df_historical_data['away'] = df_historical_data['away'].str.strip()

In [23]:
df_historical_data.head()

,home,away,score,year
0,France,Mexico,4–1,1930
1,Argentina,France,1–0,1930
2,Chile,Mexico,3–0,1930
3,Chile,France,1–0,1930
4,Argentina,Mexico,6–3,1930


In [24]:
# splitting score columns into home and away goals and dropping score column
df_historical_data[['HomeGoals', 'AwayGoals']] = df_historical_data['score'].str.split('–', expand=True)
df_historical_data.drop('score', axis=1, inplace=True)

In [25]:
df_historical_data.head()

,home,away,year,HomeGoals,AwayGoals
0,France,Mexico,1930,4,1
1,Argentina,France,1930,1,0
2,Chile,Mexico,1930,3,0
3,Chile,France,1930,1,0
4,Argentina,Mexico,1930,6,3


In [26]:
# renaming columns and changing data types
df_historical_data.rename(columns={'home': 'HomeTeam', 'away': 'AwayTeam', 
                                    'year':'Year'}, inplace=True)
df_historical_data = df_historical_data.astype({'HomeGoals': int, 'AwayGoals':int, 'Year': int})

# creating new column "totalgoals"
df_historical_data['TotalGoals'] = df_historical_data['HomeGoals'] + df_historical_data['AwayGoals']
df_historical_data

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
1,Argentina,France,1930,1,0,1
2,Chile,Mexico,1930,3,0,3
3,Chile,France,1930,1,0,1
4,Argentina,Mexico,1930,6,3,9
...,...,...,...,...,...,...
522,Russia,Croatia,2018,2,2,4
523,France,Belgium,2018,1,0,1
524,Croatia,England,2018,2,1,3
525,Belgium,England,2018,2,0,2


In [28]:
df_historical_data.dtypes

HomeTeam      object
AwayTeam      object
Year           int32
HomeGoals      int32
AwayGoals      int32
TotalGoals     int32
dtype: object

# Exporting clean dataframes

In [16]:
df_historical_data.to_csv('CleanData/clean_fifa_worldcup_matches.csv',index=False)
df_fixture.to_csv('CleanData/clean_fifa_worldcup_fixture.csv',index=False)

# Extra verifications

In [31]:
# verify number of matches per competition
years = df_historical_data['Year'].unique()

for year in years:
    print(year, len(df_historical_data[df_historical_data['Year']==year]))

1930 18
1934 17
1938 18
1950 22
1958 35
1962 32
1966 32
1970 8
1974 2
1978 2
1982 4
1986 16
1990 16
1994 16
1998 16
2002 64
2006 16
2010 64
2014 64
2018 64


In [30]:
# verify data collected for a team
print(df_historical_data[df_historical_data['HomeTeam'].str.contains('Turkey')])
print(df_historical_data[df_historical_data['AwayTeam'].str.contains('Turkey')])

    HomeTeam AwayTeam  Year  HomeGoals  AwayGoals  TotalGoals
272   Turkey    China  2002          3          0           3
        HomeTeam AwayTeam  Year  HomeGoals  AwayGoals  TotalGoals
267       Brazil   Turkey  2002          2          1           3
270   Costa Rica   Turkey  2002          1          1           2
309        Japan   Turkey  2002          0          1           1
314      Senegal   Turkey  2002          0          1           1
316       Brazil   Turkey  2002          1          0           1
317  South Korea   Turkey  2002          2          3           5
